In [1]:
%config InlineBackend.figure_format='retina'

# Add the parent directory to the path so that we can import the modules.
import os
import sys
from functools import partial

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

import matplotlib.pyplot as plt
import pandas as pd

from dynamic_contagion import *
from market_conditions import *
from reinsurance import *

In [2]:
# Other parameters
maturity = 3
markup = 0.4
R = int(1e5)
seed = 123

# Asset parameters
V_0 = 130
phi_V = -3  # * (1.3) # = V0 / L0
sigma_V = 0.05

# Liability parameters
L_0 = 100
phi_L = -3
sigma_L = 0.02

# Interest rate parameters
r_0 = 0.02
k = 0.2
m = 0.05  # 0.5
upsilon = 0.1
eta_r = -0.01

# Catastrophe loss size distribution parameters
mu_C = 2
sigma_C = 0.5

In [3]:
# Poisson process
def simulate_poisson(seed):
    lambda_ = 0.5
    rg = rnd.default_rng(seed)
    return rg.poisson(lambda_ * maturity)


# Cox proces
def simulate_cox(seed):
    lambda0 = 0.49
    a = 0.4
    rho = 0.4
    delta = 1

    return simulate_num_dynamic_contagion_uniform_jumps(
        seed, maturity, lambda0, a, rho, delta, 0.0, 0.0, 0.0, 0.5
    )


# Hawkes process
def simulate_hawkes(seed):
    lambda0 = 0.47
    a = 0.26
    rho = 0.4
    delta = 1

    return simulate_num_dynamic_contagion_uniform_jumps(
        seed, maturity, lambda0, a, rho, delta, 0.0, 1.0, 0.0, 0.0
    )


# Dynamic contagion process
def simulate_dcp(seed):
    lambda0 = 0.29
    a = 0.26
    rho = 0.4
    delta = 1

    return simulate_num_dynamic_contagion_uniform_jumps(
        seed, maturity, lambda0, a, rho, delta, 0.0, 1.0, 0.0, 0.5
    )


simulators = (simulate_poisson, simulate_cox, simulate_hawkes, simulate_dcp)

## Tables 1-4

In [4]:
ROUNDING = 4

As = (10.0, 15.0, 20.0, 25.0, 30.0)
Ms = (60.0, 65.0, 70.0, 75.0, 80.0, 85.0, 90.0)


def prices_to_df(prices, As, Ms):
    try:
        cols = [f"$M={int(m)}$" for m in Ms]
        rows = [f"$A={int(a)}$" for a in As]
    except ValueError:
        cols = [f"$M={m}$" for m in Ms]
        rows = [f"$A={a}$" for a in As]

    return pd.DataFrame(prices.round(ROUNDING), columns=cols, index=rows)


def prices_to_tex(prices, As, Ms):
    df = prices_to_df(prices, As, Ms)
    display(df)
    return (
        df.style.to_latex().replace("lrrrrrrr", "c|c|c|c|c|c|c|c").replace("00 ", " ")
    )

In [5]:
%%time
prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    V_0,
    L_0,
    r_0,
    simulators,
    mu_C,
    sigma_C,
    markup,
    A=As,
    M=Ms,
)

CPU times: total: 2.02 s
Wall time: 4.65 s


In [6]:
# Tables 1 to 4
cat_models = ("Poisson", "Cox", "Hawkes", "DCP")
for c in range(4):
    print(cat_models[c])
    print(prices_to_tex(prices[c], As, Ms))

Poisson


,$M=60$,$M=65$,$M=70$,$M=75$,$M=80$,$M=85$,$M=90$
$A=10$,7.2478,7.2561,7.2605,7.2625,7.2635,7.2640,7.2643
$A=15$,4.5758,4.5848,4.5895,4.5916,4.5927,4.5932,4.5935
$A=20$,2.7822,2.7921,2.7972,2.7995,2.8006,2.8012,2.8015
$A=25$,1.6393,1.6500,1.6557,1.6581,1.6594,1.6600,1.6603
$A=30$,0.9403,0.9519,0.9580,0.9607,0.9620,0.9627,0.9630


\begin{tabular}{c|c|c|c|c|c|c|c}
 & $M=60$ & $M=65$ & $M=70$ & $M=75$ & $M=80$ & $M=85$ & $M=90$ \\
$A=10$ & 7.2478 & 7.2561 & 7.2605 & 7.2625 & 7.2635 & 7.2640 & 7.2643 \\
$A=15$ & 4.5758 & 4.5848 & 4.5895 & 4.5916 & 4.5927 & 4.5932 & 4.5935 \\
$A=20$ & 2.7822 & 2.7921 & 2.7972 & 2.7995 & 2.8006 & 2.8012 & 2.8015 \\
$A=25$ & 1.6393 & 1.6500 & 1.6557 & 1.6581 & 1.6594 & 1.6600 & 1.6603 \\
$A=30$ & 0.9403 & 0.9519 & 0.9580 & 0.9607 & 0.9620 & 0.9627 & 0.9630 \\
\end{tabular}

Cox


,$M=60$,$M=65$,$M=70$,$M=75$,$M=80$,$M=85$,$M=90$
$A=10$,7.2451,7.2538,7.2583,7.2607,7.2620,7.2626,7.2629
$A=15$,4.6032,4.6126,4.6175,4.6201,4.6215,4.6221,4.6224
$A=20$,2.8205,2.8309,2.8362,2.8390,2.8405,2.8412,2.8415
$A=25$,1.6718,1.6831,1.6890,1.6920,1.6936,1.6944,1.6947
$A=30$,0.9628,0.9750,0.9813,0.9846,0.9863,0.9872,0.9876


\begin{tabular}{c|c|c|c|c|c|c|c}
 & $M=60$ & $M=65$ & $M=70$ & $M=75$ & $M=80$ & $M=85$ & $M=90$ \\
$A=10$ & 7.2451 & 7.2538 & 7.2583 & 7.2607 & 7.2620 & 7.2626 & 7.2629 \\
$A=15$ & 4.6032 & 4.6126 & 4.6175 & 4.6201 & 4.6215 & 4.6221 & 4.6224 \\
$A=20$ & 2.8205 & 2.8309 & 2.8362 & 2.8390 & 2.8405 & 2.8412 & 2.8415 \\
$A=25$ & 1.6718 & 1.6831 & 1.6890 & 1.6920 & 1.6936 & 1.6944 & 1.6947 \\
$A=30$ & 0.9628 & 0.9750 & 0.9813 & 0.9846 & 0.9863 & 0.9872 & 0.9876 \\
\end{tabular}

Hawkes


,$M=60$,$M=65$,$M=70$,$M=75$,$M=80$,$M=85$,$M=90$
$A=10$,8.1183,8.1950,8.2488,8.2876,8.3151,8.3339,8.3476
$A=15$,5.9402,6.0235,6.0817,6.1233,6.1525,6.1726,6.1871
$A=20$,4.3240,4.4150,4.4781,4.5231,4.5545,4.5758,4.5912
$A=25$,3.1259,3.2249,3.2938,3.3428,3.3766,3.3995,3.4158
$A=30$,2.2317,2.3390,2.4140,2.4674,2.5043,2.5289,2.5464


\begin{tabular}{c|c|c|c|c|c|c|c}
 & $M=60$ & $M=65$ & $M=70$ & $M=75$ & $M=80$ & $M=85$ & $M=90$ \\
$A=10$ & 8.1183 & 8.1950 & 8.2488 & 8.2876 & 8.3151 & 8.3339 & 8.3476 \\
$A=15$ & 5.9402 & 6.0235 & 6.0817 & 6.1233 & 6.1525 & 6.1726 & 6.1871 \\
$A=20$ & 4.3240 & 4.4150 & 4.4781 & 4.5231 & 4.5545 & 4.5758 & 4.5912 \\
$A=25$ & 3.1259 & 3.2249 & 3.2938 & 3.3428 & 3.3766 & 3.3995 & 3.4158 \\
$A=30$ & 2.2317 & 2.3390 & 2.4140 & 2.4674 & 2.5043 & 2.5289 & 2.5464 \\
\end{tabular}

DCP


,$M=60$,$M=65$,$M=70$,$M=75$,$M=80$,$M=85$,$M=90$
$A=10$,8.2255,8.3003,8.3523,8.3891,8.4147,8.4329,8.4457
$A=15$,6.0212,6.1026,6.1588,6.1981,6.2253,6.2447,6.2582
$A=20$,4.3848,4.4738,4.5349,4.5773,4.6064,4.6270,4.6414
$A=25$,3.1672,3.2644,3.3310,3.3771,3.4085,3.4305,3.4458
$A=30$,2.2562,2.3615,2.4342,2.4843,2.5184,2.5421,2.5585


\begin{tabular}{c|c|c|c|c|c|c|c}
 & $M=60$ & $M=65$ & $M=70$ & $M=75$ & $M=80$ & $M=85$ & $M=90$ \\
$A=10$ & 8.2255 & 8.3003 & 8.3523 & 8.3891 & 8.4147 & 8.4329 & 8.4457 \\
$A=15$ & 6.0212 & 6.1026 & 6.1588 & 6.1981 & 6.2253 & 6.2447 & 6.2582 \\
$A=20$ & 4.3848 & 4.4738 & 4.5349 & 4.5773 & 4.6064 & 4.6270 & 4.6414 \\
$A=25$ & 3.1672 & 3.2644 & 3.3310 & 3.3771 & 3.4085 & 3.4305 & 3.4458 \\
$A=30$ & 2.2562 & 2.3615 & 2.4342 & 2.4843 & 2.5184 & 2.5421 & 2.5585 \\
\end{tabular}



## Table 5: Default risk premium

In [7]:
%%time
risky_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    V_0,
    L_0,
    r_0,
    simulators,
    mu_C,
    sigma_C,
    markup,
)[:, np.newaxis]

CPU times: total: 1.66 s
Wall time: 3.89 s


In [8]:
risky_prices.round(4)

array([[2.8015],
       [2.8415],
       [4.5912],
       [4.6414]])

In [9]:
%%time
safe_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    V_0,
    L_0,
    r_0,
    simulators,
    mu_C,
    sigma_C,
    markup,
    defaultable=False,
)[:, np.newaxis]

CPU times: total: 1.59 s
Wall time: 4.02 s


In [10]:
safe_prices.round(4)

array([[2.847 ],
       [2.8844],
       [4.8709],
       [4.9161]])

In [11]:
risk_premium = safe_prices - risky_prices
risk_premium.round(4)

array([[0.0455],
       [0.0429],
       [0.2797],
       [0.2747]])

In [12]:
default_risk_premiums = np.hstack((risky_prices, safe_prices, risk_premium)).round(4)
default_risk_premiums

array([[2.8015, 2.847 , 0.0455],
       [2.8415, 2.8844, 0.0429],
       [4.5912, 4.8709, 0.2797],
       [4.6414, 4.9161, 0.2747]])

In [13]:
print(pd.DataFrame(default_risk_premiums).style.to_latex().replace("00 ", " "))

\begin{tabular}{lrrr}
 & 0 & 1 & 2 \\
0 & 2.8015 & 2.8470 & 0.0455 \\
1 & 2.8415 & 2.8844 & 0.0429 \\
2 & 4.5912 & 4.8709 & 0.2797 \\
3 & 4.6414 & 4.9161 & 0.2747 \\
\end{tabular}



## Table 6: Impacts of externally-excited jump frequency rate

In [14]:
def simulate_dcp_variations(seed, rho):
    lambda0 = 0.29
    a = 0.26
    delta = 1

    return simulate_num_dynamic_contagion_uniform_jumps(
        seed, maturity, lambda0, a, rho, delta, 0.0, 1.0, 0.0, 0.5
    )


dcp_variations = [partial(simulate_dcp_variations, rho=rho) for rho in (0.4, 3, 10, 20)]

In [15]:
%%time
risky_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    tuple(int(scale * L_0) for scale in (1.1, 1.3, 1.5)),
    L_0,
    r_0,
    dcp_variations,
    mu_C,
    sigma_C,
    markup,
).T

CPU times: total: 3.89 s
Wall time: 10.4 s


In [16]:
risky_prices.round(4)

array([[ 4.1942,  4.6414,  4.8367],
       [13.593 , 15.3399, 16.2959],
       [39.9647, 46.4996, 51.4606],
       [56.7379, 66.8978, 76.0682]])

In [17]:
%%time
safe_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    tuple(int(scale * L_0) for scale in (1.1, 1.3, 1.5)),
    L_0,
    r_0,
    dcp_variations,
    mu_C,
    sigma_C,
    markup,
    defaultable=False,
).T

CPU times: total: 844 ms
Wall time: 3.51 s


In [18]:
safe_prices.round(4)

array([[ 4.9161,  4.9161,  4.9161],
       [16.7538, 16.7538, 16.7538],
       [54.7041, 54.7041, 54.7041],
       [83.286 , 83.286 , 83.286 ]])

In [19]:
risk_premium = safe_prices - risky_prices

In [20]:
impacts_of_ext_jump_freq = np.hstack([risky_prices, risk_premium]).round(4)
impacts_of_ext_jump_freq

array([[ 4.1942,  4.6414,  4.8367,  0.7219,  0.2747,  0.0794],
       [13.593 , 15.3399, 16.2959,  3.1608,  1.414 ,  0.4579],
       [39.9647, 46.4996, 51.4606, 14.7394,  8.2045,  3.2435],
       [56.7379, 66.8978, 76.0682, 26.5481, 16.3882,  7.2178]])

In [21]:
print(pd.DataFrame(impacts_of_ext_jump_freq).style.to_latex().replace("00 ", " "))

\begin{tabular}{lrrrrrr}
 & 0 & 1 & 2 & 3 & 4 & 5 \\
0 & 4.1942 & 4.6414 & 4.8367 & 0.7219 & 0.2747 & 0.0794 \\
1 & 13.5930 & 15.3399 & 16.2959 & 3.1608 & 1.4140 & 0.4579 \\
2 & 39.9647 & 46.4996 & 51.4606 & 14.7394 & 8.2045 & 3.2435 \\
3 & 56.7379 & 66.8978 & 76.0682 & 26.5481 & 16.3882 & 7.2178 \\
\end{tabular}



## Table 7: Impacts of externally-excited jump magnitude

In [22]:
def simulate_dcp_variations(seed, mu_F=0.25, mu_G=0.5):
    lambda0 = 0.29
    a = 0.26
    delta = 1
    rho = 0.4

    return simulate_num_dynamic_contagion_uniform_jumps(
        seed, maturity, lambda0, a, rho, delta, 0.0, 2 * mu_G, 0.0, 2 * mu_F
    )


dcp_variations = [
    partial(simulate_dcp_variations, mu_F=mu_F) for mu_F in (0.25, 1.0, 4.0, 8.0)
]

In [23]:
%%time
risky_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    tuple(int(scale * L_0) for scale in (1.1, 1.3, 1.5)),
    L_0,
    r_0,
    dcp_variations,
    mu_C,
    sigma_C,
    markup,
).T

safe_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    tuple(int(scale * L_0) for scale in (1.1, 1.3, 1.5)),
    L_0,
    r_0,
    dcp_variations,
    mu_C,
    sigma_C,
    markup,
    defaultable=False,
).T

risk_premium = safe_prices - risky_prices

CPU times: total: 5.78 s
Wall time: 10.8 s


In [24]:
impacts_of_ext_jump_mag = np.hstack([risky_prices, risk_premium]).round(4)
impacts_of_ext_jump_mag

array([[ 4.1942,  4.6414,  4.8367,  0.7219,  0.2747,  0.0794],
       [ 8.599 ,  9.6724, 10.253 ,  1.9358,  0.8625,  0.2818],
       [23.4755, 27.2457, 30.1638,  8.7104,  4.9402,  2.0221],
       [31.6212, 37.0257, 41.6676, 13.6175,  8.213 ,  3.5712]])

In [25]:
print(pd.DataFrame(impacts_of_ext_jump_mag).style.to_latex().replace("00 ", " "))

\begin{tabular}{lrrrrrr}
 & 0 & 1 & 2 & 3 & 4 & 5 \\
0 & 4.1942 & 4.6414 & 4.8367 & 0.7219 & 0.2747 & 0.0794 \\
1 & 8.5990 & 9.6724 & 10.2530 & 1.9358 & 0.8625 & 0.2818 \\
2 & 23.4755 & 27.2457 & 30.1638 & 8.7104 & 4.9402 & 2.0221 \\
3 & 31.6212 & 37.0257 & 41.6676 & 13.6175 & 8.2130 & 3.5712 \\
\end{tabular}



## Table 8: Impacts of self-excited jump magnitude

In [26]:
%%time
dcp_variations = [
    partial(simulate_dcp_variations, mu_G=mu_G) for mu_G in (0.5, 1.0, 2.0, 3.0)
]

risky_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    tuple(int(scale * L_0) for scale in (1.1, 1.3, 1.5)),
    L_0,
    r_0,
    dcp_variations,
    mu_C,
    sigma_C,
    markup,
).T

safe_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    tuple(int(scale * L_0) for scale in (1.1, 1.3, 1.5)),
    L_0,
    r_0,
    dcp_variations,
    mu_C,
    sigma_C,
    markup,
    defaultable=False,
).T

risk_premium = safe_prices - risky_prices

CPU times: total: 5.62 s
Wall time: 13.3 s


In [27]:
impact_of_self_jump_mag = np.hstack([risky_prices, risk_premium]).round(4)
impact_of_self_jump_mag

array([[ 4.1942,  4.6414,  4.8367,  0.7219,  0.2747,  0.0794],
       [ 9.1782, 10.4914, 11.379 ,  2.7508,  1.4376,  0.5499],
       [19.0579, 22.2719, 25.0156,  8.0597,  4.8457,  2.102 ],
       [24.9325, 29.305 , 33.245 , 11.4795,  7.1069,  3.1669]])

In [28]:
print(pd.DataFrame(impact_of_self_jump_mag).style.to_latex().replace("00 ", " "))

\begin{tabular}{lrrrrrr}
 & 0 & 1 & 2 & 3 & 4 & 5 \\
0 & 4.1942 & 4.6414 & 4.8367 & 0.7219 & 0.2747 & 0.0794 \\
1 & 9.1782 & 10.4914 & 11.3790 & 2.7508 & 1.4376 & 0.5499 \\
2 & 19.0579 & 22.2719 & 25.0156 & 8.0597 & 4.8457 & 2.1020 \\
3 & 24.9325 & 29.3050 & 33.2450 & 11.4795 & 7.1069 & 3.1669 \\
\end{tabular}



## Tables 9-12: Prices with catbond

In [29]:
K = 40.0
F = 25.0

In [30]:
%%time
catbond_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    V_0,
    L_0,
    r_0,
    simulators,
    mu_C,
    sigma_C,
    markup,
    A=As,
    M=Ms,
    catbond=True,
    K=K,
    F=F,
)

CPU times: total: 1.66 s
Wall time: 3.91 s


In [31]:
print("Prices of reinsurance when the reinsurer issues a catbond\n")

for c in range(4):
    print(cat_models[c])
    print(prices_to_tex(catbond_prices[c], As, Ms))

Prices of reinsurance when the reinsurer issues a catbond

Poisson


,$M=60$,$M=65$,$M=70$,$M=75$,$M=80$,$M=85$,$M=90$
$A=10$,7.3011,7.3128,7.3189,7.3216,7.3229,7.3235,7.3239
$A=15$,4.6137,4.6261,4.6328,4.6357,4.6371,4.6378,4.6382
$A=20$,2.8065,2.8195,2.8266,2.8298,2.8314,2.8321,2.8325
$A=25$,1.6530,1.6664,1.6738,1.6773,1.6790,1.6799,1.6803
$A=30$,0.9472,0.9606,0.9683,0.9720,0.9738,0.9748,0.9753


\begin{tabular}{c|c|c|c|c|c|c|c}
 & $M=60$ & $M=65$ & $M=70$ & $M=75$ & $M=80$ & $M=85$ & $M=90$ \\
$A=10$ & 7.3011 & 7.3128 & 7.3189 & 7.3216 & 7.3229 & 7.3235 & 7.3239 \\
$A=15$ & 4.6137 & 4.6261 & 4.6328 & 4.6357 & 4.6371 & 4.6378 & 4.6382 \\
$A=20$ & 2.8065 & 2.8195 & 2.8266 & 2.8298 & 2.8314 & 2.8321 & 2.8325 \\
$A=25$ & 1.6530 & 1.6664 & 1.6738 & 1.6773 & 1.6790 & 1.6799 & 1.6803 \\
$A=30$ & 0.9472 & 0.9606 & 0.9683 & 0.9720 & 0.9738 & 0.9748 & 0.9753 \\
\end{tabular}

Cox


,$M=60$,$M=65$,$M=70$,$M=75$,$M=80$,$M=85$,$M=90$
$A=10$,7.2977,7.3101,7.3165,7.3198,7.3215,7.3223,7.3227
$A=15$,4.6397,4.6527,4.6597,4.6634,4.6652,4.6661,4.6665
$A=20$,2.8430,2.8567,2.8640,2.8680,2.8701,2.8711,2.8715
$A=25$,1.6840,1.6980,1.7057,1.7099,1.7121,1.7133,1.7137
$A=30$,0.9687,0.9827,0.9906,0.9950,0.9974,0.9986,0.9991


\begin{tabular}{c|c|c|c|c|c|c|c}
 & $M=60$ & $M=65$ & $M=70$ & $M=75$ & $M=80$ & $M=85$ & $M=90$ \\
$A=10$ & 7.2977 & 7.3101 & 7.3165 & 7.3198 & 7.3215 & 7.3223 & 7.3227 \\
$A=15$ & 4.6397 & 4.6527 & 4.6597 & 4.6634 & 4.6652 & 4.6661 & 4.6665 \\
$A=20$ & 2.8430 & 2.8567 & 2.8640 & 2.8680 & 2.8701 & 2.8711 & 2.8715 \\
$A=25$ & 1.6840 & 1.6980 & 1.7057 & 1.7099 & 1.7121 & 1.7133 & 1.7137 \\
$A=30$ & 0.9687 & 0.9827 & 0.9906 & 0.9950 & 0.9974 & 0.9986 & 0.9991 \\
\end{tabular}

Hawkes


,$M=60$,$M=65$,$M=70$,$M=75$,$M=80$,$M=85$,$M=90$
$A=10$,8.3431,8.4519,8.5285,8.5820,8.6187,8.6429,8.6598
$A=15$,6.1037,6.2194,6.3021,6.3613,6.4016,6.4283,6.4468
$A=20$,4.4321,4.5526,4.6404,4.7043,4.7490,4.7784,4.7988
$A=25$,3.1898,3.3130,3.4044,3.4724,3.5207,3.5533,3.5759
$A=30$,2.2650,2.3896,2.4829,2.5536,2.6050,2.6403,2.6655


\begin{tabular}{c|c|c|c|c|c|c|c}
 & $M=60$ & $M=65$ & $M=70$ & $M=75$ & $M=80$ & $M=85$ & $M=90$ \\
$A=10$ & 8.3431 & 8.4519 & 8.5285 & 8.5820 & 8.6187 & 8.6429 & 8.6598 \\
$A=15$ & 6.1037 & 6.2194 & 6.3021 & 6.3613 & 6.4016 & 6.4283 & 6.4468 \\
$A=20$ & 4.4321 & 4.5526 & 4.6404 & 4.7043 & 4.7490 & 4.7784 & 4.7988 \\
$A=25$ & 3.1898 & 3.3130 & 3.4044 & 3.4724 & 3.5207 & 3.5533 & 3.5759 \\
$A=30$ & 2.2650 & 2.3896 & 2.4829 & 2.5536 & 2.6050 & 2.6403 & 2.6655 \\
\end{tabular}

DCP


,$M=60$,$M=65$,$M=70$,$M=75$,$M=80$,$M=85$,$M=90$
$A=10$,8.4492,8.5565,8.6307,8.6811,8.7147,8.7378,8.7535
$A=15$,6.1830,6.2966,6.3774,6.4334,6.4707,6.4960,6.5132
$A=20$,4.4913,4.6094,4.6950,4.7561,4.7975,4.8258,4.8444
$A=25$,3.2302,3.3509,3.4398,3.5046,3.5498,3.5811,3.6020
$A=30$,2.2892,2.4112,2.5021,2.5693,2.6172,2.6514,2.6747


\begin{tabular}{c|c|c|c|c|c|c|c}
 & $M=60$ & $M=65$ & $M=70$ & $M=75$ & $M=80$ & $M=85$ & $M=90$ \\
$A=10$ & 8.4492 & 8.5565 & 8.6307 & 8.6811 & 8.7147 & 8.7378 & 8.7535 \\
$A=15$ & 6.1830 & 6.2966 & 6.3774 & 6.4334 & 6.4707 & 6.4960 & 6.5132 \\
$A=20$ & 4.4913 & 4.6094 & 4.6950 & 4.7561 & 4.7975 & 4.8258 & 4.8444 \\
$A=25$ & 3.2302 & 3.3509 & 3.4398 & 3.5046 & 3.5498 & 3.5811 & 3.6020 \\
$A=30$ & 2.2892 & 2.4112 & 2.5021 & 2.5693 & 2.6172 & 2.6514 & 2.6747 \\
\end{tabular}



## Tables 13-16: Percentage increase in costs after issuing a catbond

In [32]:
print(
    "Percentage increase in price for reinsurance after the reinsurer issues a catbond\n"
)

for c in range(4):
    print(cat_models[c])
    percentage_increase = ((catbond_prices[c] - prices[c]) / prices[c] * 100).round(2)
    print(prices_to_tex(percentage_increase, As, Ms).replace("00 ", "\% "))

Percentage increase in price for reinsurance after the reinsurer issues a catbond

Poisson


,$M=60$,$M=65$,$M=70$,$M=75$,$M=80$,$M=85$,$M=90$
$A=10$,0.74,0.78,0.80,0.81,0.82,0.82,0.82
$A=15$,0.83,0.90,0.94,0.96,0.97,0.97,0.97
$A=20$,0.87,0.98,1.05,1.08,1.10,1.10,1.11
$A=25$,0.84,0.99,1.10,1.15,1.18,1.20,1.20
$A=30$,0.72,0.92,1.08,1.17,1.23,1.26,1.27


\begin{tabular}{c|c|c|c|c|c|c|c}
 & $M=60$ & $M=65$ & $M=70$ & $M=75$ & $M=80$ & $M=85$ & $M=90$ \\
$A=10$ & 0.74\% & 0.78\% & 0.80\% & 0.81\% & 0.82\% & 0.82\% & 0.82\% \\
$A=15$ & 0.83\% & 0.90\% & 0.94\% & 0.96\% & 0.97\% & 0.97\% & 0.97\% \\
$A=20$ & 0.87\% & 0.98\% & 1.05\% & 1.08\% & 1.10\% & 1.10\% & 1.11\% \\
$A=25$ & 0.84\% & 0.99\% & 1.10\% & 1.15\% & 1.18\% & 1.20\% & 1.20\% \\
$A=30$ & 0.72\% & 0.92\% & 1.08\% & 1.17\% & 1.23\% & 1.26\% & 1.27\% \\
\end{tabular}

Cox


,$M=60$,$M=65$,$M=70$,$M=75$,$M=80$,$M=85$,$M=90$
$A=10$,0.73,0.78,0.80,0.81,0.82,0.82,0.82
$A=15$,0.79,0.87,0.91,0.94,0.95,0.95,0.95
$A=20$,0.80,0.91,0.98,1.02,1.04,1.05,1.05
$A=25$,0.73,0.88,0.99,1.06,1.09,1.11,1.12
$A=30$,0.61,0.79,0.94,1.05,1.12,1.15,1.17


\begin{tabular}{c|c|c|c|c|c|c|c}
 & $M=60$ & $M=65$ & $M=70$ & $M=75$ & $M=80$ & $M=85$ & $M=90$ \\
$A=10$ & 0.73\% & 0.78\% & 0.80\% & 0.81\% & 0.82\% & 0.82\% & 0.82\% \\
$A=15$ & 0.79\% & 0.87\% & 0.91\% & 0.94\% & 0.95\% & 0.95\% & 0.95\% \\
$A=20$ & 0.80\% & 0.91\% & 0.98\% & 1.02\% & 1.04\% & 1.05\% & 1.05\% \\
$A=25$ & 0.73\% & 0.88\% & 0.99\% & 1.06\% & 1.09\% & 1.11\% & 1.12\% \\
$A=30$ & 0.61\% & 0.79\% & 0.94\% & 1.05\% & 1.12\% & 1.15\% & 1.17\% \\
\end{tabular}

Hawkes


,$M=60$,$M=65$,$M=70$,$M=75$,$M=80$,$M=85$,$M=90$
$A=10$,2.77,3.14,3.39,3.55,3.65,3.71,3.74
$A=15$,2.75,3.25,3.62,3.89,4.05,4.14,4.20
$A=20$,2.50,3.12,3.62,4.01,4.27,4.43,4.52
$A=25$,2.05,2.73,3.36,3.88,4.27,4.52,4.69
$A=30$,1.49,2.17,2.85,3.50,4.02,4.41,4.67


\begin{tabular}{c|c|c|c|c|c|c|c}
 & $M=60$ & $M=65$ & $M=70$ & $M=75$ & $M=80$ & $M=85$ & $M=90$ \\
$A=10$ & 2.77\% & 3.14\% & 3.39\% & 3.55\% & 3.65\% & 3.71\% & 3.74\% \\
$A=15$ & 2.75\% & 3.25\% & 3.62\% & 3.89\% & 4.05\% & 4.14\% & 4.20\% \\
$A=20$ & 2.50\% & 3.12\% & 3.62\% & 4.01\% & 4.27\% & 4.43\% & 4.52\% \\
$A=25$ & 2.05\% & 2.73\% & 3.36\% & 3.88\% & 4.27\% & 4.52\% & 4.69\% \\
$A=30$ & 1.49\% & 2.17\% & 2.85\% & 3.50\% & 4.02\% & 4.41\% & 4.67\% \\
\end{tabular}

DCP


,$M=60$,$M=65$,$M=70$,$M=75$,$M=80$,$M=85$,$M=90$
$A=10$,2.72,3.09,3.33,3.48,3.56,3.62,3.64
$A=15$,2.69,3.18,3.55,3.80,3.94,4.02,4.07
$A=20$,2.43,3.03,3.53,3.91,4.15,4.29,4.38
$A=25$,1.99,2.65,3.27,3.78,4.15,4.39,4.53
$A=30$,1.46,2.11,2.79,3.42,3.92,4.30,4.54


\begin{tabular}{c|c|c|c|c|c|c|c}
 & $M=60$ & $M=65$ & $M=70$ & $M=75$ & $M=80$ & $M=85$ & $M=90$ \\
$A=10$ & 2.72\% & 3.09\% & 3.33\% & 3.48\% & 3.56\% & 3.62\% & 3.64\% \\
$A=15$ & 2.69\% & 3.18\% & 3.55\% & 3.80\% & 3.94\% & 4.02\% & 4.07\% \\
$A=20$ & 2.43\% & 3.03\% & 3.53\% & 3.91\% & 4.15\% & 4.29\% & 4.38\% \\
$A=25$ & 1.99\% & 2.65\% & 3.27\% & 3.78\% & 4.15\% & 4.39\% & 4.53\% \\
$A=30$ & 1.46\% & 2.11\% & 2.79\% & 3.42\% & 3.92\% & 4.30\% & 4.54\% \\
\end{tabular}

